In [1]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import sklearn.metrics as skm
import wandb
import torch

from torch import nn
from torch.utils.data import TensorDataset, DataLoader
from tsai.all import *
from fastai.callback.wandb import *
from visualization_functions import *

In [2]:
computer_setup()

os             : Linux-5.11.0-38-generic-x86_64-with-debian-bullseye-sid
python         : 3.7.12
tsai           : 0.3.1
fastai         : 2.6.3
fastcore       : 1.4.3
torch          : 1.11.0+cu102
device         : 4 gpus (['NVIDIA GeForce RTX 2080 Ti', 'NVIDIA GeForce RTX 2080 Ti', 'NVIDIA GeForce RTX 2080 Ti', 'NVIDIA GeForce RTX 2080 Ti'])
cpu cores      : 16
RAM            : 62.49 GB
GPU memory     : [10.76, 10.76, 10.76, 10.76] GB


## Load Raw Data

In [3]:
DATA_ALL = sio.loadmat("subjects_40_v4.mat")
# DATA_ALL = sio.loadmat("subjects_88_v2.mat")
VFI_LIST = sio.loadmat("Notes/VFI_new_matched.mat")

VFI_1       = np.ravel(VFI_LIST['VFI1'])
VFI_2       = np.ravel(VFI_LIST['VFI2'])
SIG         = DATA_ALL['DATA']              # raw sEMG signals
FEAT        = DATA_ALL['FEAT']              # Orignally calculated features
FEAT_N      = DATA_ALL['FEAT_N']            # Normalized features
LABEL       = DATA_ALL['LABEL']             # Labels
SUBJECT_ID  = DATA_ALL['SUBJECT_ID']        # Sujbect ID
LABEL_VOWEL = DATA_ALL['LABEL_VOWEL']
VFI         = DATA_ALL['SUBJECT_VFI']

In [4]:
wand_config = 0
leftout = 1
window_length = 4000
testing_acc = np.zeros(40)

In [5]:
# sub_index = [0,1,2,3,4,20,21,22,23,24]
# for sub_test in sub_index:
# for sub_test in range(40):

sub_test = 3

sub_txt = "R%03d"%(int(SUBJECT_ID[sub_test][0][0]))
print('Test Subject %s:'%(sub_txt))
print('VFI-1:', (VFI[sub_test][0][0]))
if int(VFI[sub_test][0][0]) > 10:
    sub_group = 'Fatigued'
else:
    sub_group = 'Healthy'

if wand_config == 1:
    run = wandb.init(project="sEMG Leave-One-Out Classification 40 0.9 Left-Out", 
                     group=sub_group,
                     reinit=True, 
                     name=sub_txt)

# ===== Load Testing Signals =====
num_signal = np.shape(SIG[sub_test,0])[0]    
X_Temp = np.zeros((num_signal, 4, window_length))
for ch in range(4):
    X_Temp[:,ch,:] = SIG[sub_test,ch]

Y_Temp = LABEL[sub_test,0].flatten()

num_leftout = round(leftout*num_signal)
index_leftout = np.random.choice(range(num_signal), size=num_leftout, replace=False)
# index_leftout = np.random.randint(low=0, high=num_signal, size=num_leftout)
# print(np.sort(index_leftout))
print("Left-out Test samples: ", index_leftout.size)

X_Test = X_Temp[index_leftout,:,:]
Y_Test = Y_Temp[index_leftout]

index_include = np.arange(num_signal)
index_include = np.delete(index_include, index_leftout)
# print(index_include)
print("Included Training samples: ", index_include.size)
X_include = X_Temp[index_include,:,:]
Y_include = Y_Temp[index_include]


# ===== Load Traing Signals =====
X_Train = np.zeros((0,4,window_length))
Y_Train = np.zeros(0)    
for sub_train in range(40):
    if sub_train != sub_test:
        # ===== CAN BE CONVERTED INTO A FUNCTION =====            
        num_signal = np.shape(SIG[sub_train,0])[0]
        x_s = np.zeros((num_signal, 4, window_length))
        for ch in range(4):
            x_s[:,ch,:] = SIG[sub_train,ch]

        y_s = LABEL[sub_train,0].flatten()
        # ===== CAN BE CONVERTED INTO A FUNCTION =====

        X_Train = np.concatenate((X_Train, x_s), axis=0)
        Y_Train = np.concatenate((Y_Train, y_s), axis=0)

print('Train data BEFORE including:')                        
print(X_Train.shape, Y_Train.shape)

X_Train = np.concatenate((X_Train, X_include), axis=0)
Y_Train = np.concatenate((Y_Train, Y_include), axis=0)        

print('Train data AFTER including:')                
print(X_Train.shape, Y_Train.shape)
print('# of Healthy Samples: %d'%(np.sum(Y_Train == -1)))
print('# of Fatigued Samples: %d'%(np.sum(Y_Train == 1)))    

Test Subject R085:
VFI-1: [1]
Left-out Test samples:  162
Included Training samples:  0
Train data BEFORE including:
(6310, 4, 4000) (6310,)
Train data AFTER including:
(6310, 4, 4000) (6310,)
# of Healthy Samples: 3040
# of Fatigued Samples: 3270


In [7]:
# ===== Data loading and preprocessing =====
# Training and Validation
splits = get_splits(Y_Train, valid_size=.1, stratify=True, random_state=23, shuffle=True, show_plot=False)
tfms  = [None, [Categorize()]]
dsets = TSDatasets(X_Train, Y_Train, tfms=tfms, splits=splits)

# dls = TSDataLoaders.from_dsets(dsets.train, 
#                                dsets.valid, 
#                                shuffle_train=True,
#                                bs=32, 
#                                batch_tfms=TSStandardize(mean=dsets_mean,
#                                                         std=dsets_std,
#                                                         by_var=True), 
#                                num_workers=0)

dls = TSDataLoaders.from_dsets(dsets.train, 
                               dsets.valid, 
                               shuffle_train=True,
                               bs=32, 
                               batch_tfms=TSStandardize(use_single_batch=False,
                                                        by_var=True), 
                               num_workers=0)

# dls.show_batch()

# ===== FCN Classification =====
# Need to check fastai wandb setting
model = FCN(c_in=dls.vars, c_out=dls.c, layers=[64, 128, 64], kss=[7, 5, 3])
print(model)
print(dls.c)

FCN(
  (convblock1): ConvBlock(
    (0): Conv1d(4, 64, kernel_size=(7,), stride=(1,), padding=(3,), bias=False)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (convblock2): ConvBlock(
    (0): Conv1d(64, 128, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (convblock3): ConvBlock(
    (0): Conv1d(128, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (gap): GAP1d(
    (gap): AdaptiveAvgPool1d(output_size=1)
    (flatten): Flatten(full=False)
  )
  (fc): Linear(in_features=64, out_features=2, bias=True)
)
2


In [ ]:
if wand_config == 1:
    learn = Learner(dls, model, metrics=accuracy, cbs=WandbCallback())
else:
    learn = Learner(dls, model, metrics=accuracy)


with learn.no_bar():
    lr = learn.lr_find(stop_div=False, num_it=200)
    print("LR fine: %.6f"%(lr.valley))        
    print("Training...")
    learn.fit_one_cycle(250, lr_max=0.001)

# learn.recorder.plot_metrics()

# the prediction returedn by the function for some reason was in string so needed to be converted.
inp, test_probas, _, test_preds = learn.get_X_preds(X_Test, with_input=True, with_decoded=True)
# test_probas, test_targets, test_preds
# print("From get_X_preds - Input:", inp.data)
# print("From get_X_preds - Preds:", test_probas.data)

preds = np.fromstring(test_preds[1:-1], sep=',')
print(f'Testing accuracy:{skm.accuracy_score(Y_Test, preds):10.6f}\n\n')
testing_acc[sub_test] = skm.accuracy_score(Y_Test, preds)

if wand_config == 1:
    wandb.log({"VFI-1": int(VFI[sub_test][0][0]),
               "Validation Accuracy": learn.validate()[1],
               "Testing Accuracy": skm.accuracy_score(Y_Test, preds)})
    run.finish()